In [1]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
#from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

In [2]:
df = pd.read_csv('application_train.csv')
test_df = pd.read_csv('application_test.csv')
cc = pd.read_csv('credit_card_balance.csv')
pos = pd.read_csv('POS_CASH_balance.csv')
ins = pd.read_csv('installments_payments.csv')
bureau = pd.read_csv('bureau.csv')
bb = pd.read_csv('bureau_balance.csv')
prev = pd.read_csv('previous_application.csv')

In [3]:
print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
print("Credit Card: {}, POS: {}, installments samples: {}".format(len(cc), len(pos), len(ins)))
print("Bureau info: {}, balance: {}".format(len(bureau), len(bb)))

Train samples: 307511, test samples: 48744
Credit Card: 3840312, POS: 10001358, installments samples: 13605401
Bureau info: 1716428, balance: 27299925


In [72]:
df = pd.read_csv('application_train.csv')
test_df = pd.read_csv('application_test.csv')

In [73]:
def one_hot_encoder(df):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [74]:
df = df.append(test_df).reset_index()
df = df[df['CODE_GENDER'] != 'XNA']
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

# Some simple new features (percentages)
df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

#Создание признаков для следующей категоризации
df['DAYS_EMPLOYED']=abs(df['DAYS_EMPLOYED']//365)
df['DAYS_REGISTRATION']=abs(df['DAYS_REGISTRATION']//365)
df['DAYS_ID_PUBLISH']=abs(df['DAYS_ID_PUBLISH']//365)
df['age']=abs(df['DAYS_BIRTH']//365)
df['phone']=abs(df['DAYS_LAST_PHONE_CHANGE']//365)

# additional features
df['NEW_CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
df['NEW_CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
df['NEW_INC_PER_CHLD'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])
df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['NEW_ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL'])
df['NEW_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
df['NEW_EXT_SOURCES_MEAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
df['NEW_SCORES_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
df['NEW_SCORES_STD'] = df['NEW_SCORES_STD'].fillna(df['NEW_SCORES_STD'].mean())
df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
df['NEW_CAR_TO_EMPLOY_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
df['NEW_PHONE_TO_EMPLOY_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']



docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
k = df[['AMT_INCOME_TOTAL','REGION_POPULATION_RELATIVE']].groupby('REGION_POPULATION_RELATIVE').median()['AMT_INCOME_TOTAL']

df['REGION_POPULATION_RELATIVE1']=df['REGION_POPULATION_RELATIVE']*1000
df['NEW_INC_BY_RER'] = df['REGION_POPULATION_RELATIVE'].map(k)
df['NEW_INC_BY_ORG'] = df['ORGANIZATION_TYPE'].map(inc_by_org)
df['NEW_DOC_IND_AVG'] = df[docs].mean(axis=1)
df['NEW_DOC_IND_STD'] = df[docs].std(axis=1)
df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1)
df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)
df['NEW_LIVE_IND_STD'] = df[live].std(axis=1)
df['NEW_LIVE_IND_KURT'] = df[live].kurtosis(axis=1)

In [75]:
def phone_group (row):
    if 0<= row['phone'] <= 1:
        return 0

    if 1<= row['phone'] <= 2:
        return 1

    if 2<= row['phone'] <= 3:
        return 2

    if 3<= row['phone'] <= 5: 
        return 3
    
    if 5<= row['phone'] <= 13:
        return 4

    else:
        return 5


def family_group (row):
    if  row['CNT_FAM_MEMBERS'] == 0:
        return '0'

    if row['CNT_FAM_MEMBERS'] == 1:
        return '1'

    if row['CNT_FAM_MEMBERS'] == 2:
        return '2'

    if row['CNT_FAM_MEMBERS'] == 3 : 
        return '3'
    
    if row['CNT_FAM_MEMBERS'] == 4:
        return '4'

    else:
        return 'vibrosi'

def region (row):
    if 0<= row['REGION_POPULATION_RELATIVE1'] < 5:
        return 'very_lo'

    if 5<= row['REGION_POPULATION_RELATIVE1'] < 10:
        return 'lo'

    if 10<= row['REGION_POPULATION_RELATIVE1'] <= 20: 
        return 'mid'
    
    if 20<= row['REGION_POPULATION_RELATIVE1'] <= 40:
        return 'hi_mid'

    if 40<= row['REGION_POPULATION_RELATIVE1'] <= 70: 
        return 'hi'
    
    else:
        return 'very_high'
    
def age_group (row):
    if 18<= row['age'] <= 22:
        return 'very_young'

    if 23<= row['age'] <= 30:
        return 'young'

    if 30<= row['age'] <= 40:
        return 'older_young'

    if 40<= row['age'] <= 55: 
        return 'adult'
    
    if 55<= row['age'] <= 85: 
        return 'super_puper_grand'
    
    else:
        return 'vibrosi'
    
def Employ (row):
    if 0<= row['DAYS_EMPLOYED'] <= 0.5:
        return 'malo'

    if 0.5<= row['DAYS_EMPLOYED'] <= 1:
        return 'young1'

    if 1<= row['DAYS_EMPLOYED'] <= 2:
        return 'older_young1'

    if 2<= row['DAYS_EMPLOYED'] <= 5: 
        return 'adult1'
    
    if 5<= row['DAYS_EMPLOYED'] <= 13:
        return 'mnogo'

    else:
        return 'vibrosi'


def REGISTR (row):
    if 0<= row['DAYS_REGISTRATION'] <= 1:
        return 'very_young2'

    if 1<= row['DAYS_REGISTRATION'] <= 2:
        return 'young2'

    if 2<= row['DAYS_REGISTRATION'] <= 5:
        return 'older_young2'

    if 5<= row['DAYS_REGISTRATION'] <= 7: 
        return 'adult2'
    
    if 7<= row['DAYS_REGISTRATION'] <= 13:
        return 'adult_plus2'

    else:
        return 'vibrosi2'
    
def ID_PUBLISH (row):
    if 0<= row['DAYS_ID_PUBLISH'] <= 1:
        return 'very_young3'

    if 1<= row['DAYS_ID_PUBLISH'] <= 3:
        return 'young3'

    if 3<= row['DAYS_ID_PUBLISH'] <= 5:
        return 'older_young3'

    if 5<= row['DAYS_ID_PUBLISH'] <= 7: 
        return 'adult3'
    
    if 7<= row['DAYS_ID_PUBLISH'] <= 13:
        return 'adult_plus3'

    else:
        return 'vibrosi3'
    
def IsWeekend (row):
    if row['WEEKDAY_APPR_PROCESS_START'] == 'SATURDAY':
        return 1
    
    if row['WEEKDAY_APPR_PROCESS_START'] == 'SANDAY':
        return 1
    
    else:
        return 0


def education (row): 
    if row['NAME_EDUCATION_TYPE'] == 'Higher education': 
        return 1
    if row['NAME_EDUCATION_TYPE'] == 'Academic degree': 
        return 1       
    else: 
        return 0
    
def family_stat (row): 
    if row['NAME_FAMILY_STATUS'] == 'Married': 
        return 1
    if row['NAME_FAMILY_STATUS'] == 'Civil marriage': 
        return 1
   
    else: 
        return 0 

In [76]:
#df['w_h']=df['WEEKDAY_APPR_PROCESS_START'] + str(df['HOUR_APPR_PROCESS_START'])

#df['phone_group'] = df.apply(phone_group, axis=1)
#df['family_group'] = df.apply(family_group, axis=1)
#df['region_group'] = df.apply(region, axis=1)
#df['age_group'] = df.apply(age_group, axis=1)
#df['age_group'] = df.apply(age_group, axis=1)
#df['Employ'] = df.apply(Employ, axis=1)
#df['Registr'] = df.apply(REGISTR, axis=1)
#df['ID_publish'] = df.apply(ID_PUBLISH, axis=1)
#df['IsWeekend'] = df.apply(IsWeekend, axis=1)
#df['bolshe']=df.apply(asf, axis=1)
#df['car'] = df.apply(car, axis=1)
#df['realty']=df.apply(realty, axis=1)
#df['gender']=df.apply(gender, axis=1)
df['education']=df.apply(education, axis=1)
df['family_stat']=df.apply(family_stat, axis=1)

df['ext1'] = df['EXT_SOURCE_1'] + df['EXT_SOURCE_2'] +df['EXT_SOURCE_3']
df['ext2'] = (df['EXT_SOURCE_1'] + df['EXT_SOURCE_2'] +df['EXT_SOURCE_3'])/3
df['ext3'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3'] 
df['ext4'] = df['EXT_SOURCE_1'] /df['EXT_SOURCE_2'] /df['EXT_SOURCE_3'] 
df['ext5'] = df['EXT_SOURCE_1'] *df['EXT_SOURCE_2'] /df['EXT_SOURCE_3'] 
df['ext6'] = df['EXT_SOURCE_1'] +df['EXT_SOURCE_2'] -df['EXT_SOURCE_3'] 
df['ext7'] = df['EXT_SOURCE_1'] -df['EXT_SOURCE_2'] +df['EXT_SOURCE_3'] 
df['ext8'] = df['EXT_SOURCE_1'] *df['EXT_SOURCE_2']
df['ext9'] = df['EXT_SOURCE_1'] *df['EXT_SOURCE_3']
df['ext10'] = df['EXT_SOURCE_3'] *df['EXT_SOURCE_2']
df['ext11'] = df['EXT_SOURCE_1']**2
df['ext12'] = df['EXT_SOURCE_2']**2
df['ext13'] = df['EXT_SOURCE_3']**2

without_history=df[df['EXT_SOURCE_1'].isna()]

b=without_history[without_history['EXT_SOURCE_3'].isna()]

cat_feat = list(df.dtypes[df.dtypes == object].index)
num_feat = [f for f in df if f not in cat_feat ]

def get_woe_v1(df_train, df_test, col, target_col):
    all_good = len(df_train[df_train[target_col] == 1][col])
    all_bad = len(df_train[df_train[target_col] == 0][col])
    odds_series = (
        df_train[df_train[target_col] == 1][col].value_counts()
        /
        df_train[df_train[target_col] == 0][col].value_counts()
    )
    odds_series = odds_series / all_good * all_bad
    category_woe_dict = np.log(odds_series).to_dict()
    df_train[col + '_woe'] = df_train[col].apply(category_woe_dict.get)
    df_test[col + '_woe'] = df_test[col].apply(category_woe_dict.get)
    return df_train, df_test


for cat_feat in cat_feat:
    data_train, data_test = get_woe_v1(train, test, cat_feat, 'TARGET')
    
cat_feat = list(data_train.dtypes[data_train.dtypes == object].index)
num_feat = [f for f in train if f not in cat_feat ]

df=data_train.drop(cat_feat, axis=1)
df=data_test.drop(cat_feat, axis=1)

In [77]:
df.to_csv('train123.csv')

In [9]:
#Categorical features with One-Hot encode
#df, cat_cols = one_hot_encoder(df)
# Categorical features with Binary encode (0 or 1; two categories)
#for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
 #   df[bin_feature], uniques = pd.factorize(df[bin_feature])

del prev['AMT_DOWN_PAYMENT']
del prev['RATE_INTEREST_PRIMARY']
del prev['RATE_INTEREST_PRIVILEGED']
del prev['NAME_TYPE_SUITE']
del prev['DAYS_FIRST_DRAWING']
del prev['DAYS_TERMINATION']
del prev['DAYS_DECISION']
del prev['HOUR_APPR_PROCESS_START']
del prev['SELLERPLACE_AREA']
del prev['NAME_PRODUCT_TYPE']
del prev['CHANNEL_TYPE']
del prev['NAME_PAYMENT_TYPE']
del prev['PRODUCT_COMBINATION']
del prev['AMT_GOODS_PRICE']
del prev['NFLAG_LAST_APPL_IN_DAY']
del prev['NAME_CASH_LOAN_PURPOSE']
del prev['NAME_SELLER_INDUSTRY']

In [59]:

def GOOD_CAT (row):        
    if row['NAME_GOODS_CATEGORY'] == 'Mobile':
        return 'Mobile'   
    if row['NAME_GOODS_CATEGORY'] == 'Consumer Electronics':
        return 'Electronics'
    if row['NAME_GOODS_CATEGORY'] == 'Photo / Cinema Equipment':
        return 'Electronics'
    if row['NAME_GOODS_CATEGORY'] == 'Computers':
        return 'Electronics'
    if row['NAME_GOODS_CATEGORY'] == 'Audio/Video':
        return 'Electronics'     
    if row['NAME_GOODS_CATEGORY'] == 'Construction Materials':
        return '3'  
    if row['NAME_GOODS_CATEGORY'] == 'Furniture':
        return '3'
    if row['NAME_GOODS_CATEGORY'] == 'Homewares':
        return '3' 
    if row['NAME_GOODS_CATEGORY'] == 'Gardening':
        return '3'
    if row['NAME_GOODS_CATEGORY'] == 'Direct Sales':
        return '3'     
    if row['NAME_GOODS_CATEGORY'] == 'Vehicles':
        return '4'   
    if row['NAME_GOODS_CATEGORY'] == 'House Construction':
        return '4'      
    else:
        return '0'
    
    
def IsWeekend (row):
    if row['WEEKDAY_APPR_PROCESS_START'] == 'SATURDAY':
        return '1' 
    if row['WEEKDAY_APPR_PROCESS_START'] == 'SANDAY':
        return '1'
    else:
        return '0'
    
    
prev['IsWeekend'] = prev.apply(IsWeekend, axis=1)
prev['GOOD_CAT'] = prev.apply(GOOD_CAT, axis=1)

del prev['NAME_GOODS_CATEGORY']
del prev['WEEKDAY_APPR_PROCESS_START']



In [60]:

prev, cat_cols = one_hot_encoder(prev)

    # Days 365.243 values -> nan
#prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
#prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

    # Add feature: value ask / value received percentage
prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
prev['APP_ANNUTY_TO_CRED'] = prev['AMT_ANNUITY'] / prev['AMT_CREDIT']


    # Previous applications numeric features
num_aggregations = {
        'AMT_ANNUITY': ['mean', 'sum', 'min', 'max', 'median', 'std', 'size'],
        'AMT_CREDIT': ['mean', 'sum', 'min', 'max',  'median', 'std', 'size'],
        'APP_CREDIT_PERC': ['mean', 'sum', 'min', 'max',  'median', 'std', 'size'],
        'RATE_DOWN_PAYMENT': ['mean', 'sum', 'min', 'max', 'median', 'std', 'size'],
        'CNT_PAYMENT': ['mean', 'sum', 'min', 'max',  'median',  'std', 'size'],
        'AMT_DOWN_PAYMENT' : ['mean', 'sum', 'min', 'max', 'median',  'std', 'size'],
        'RATE_INTEREST_PRIVILEGED' : ['mean', 'sum', 'min', 'max',  'median',  'std', 'size'],
        'DAYS_TERMINATION' : ['mean', 'sum', 'min', 'max',  'median',  'std', 'size'],
        'AMT_GOODS_PRICE' : ['mean', 'sum', 'min', 'max', 'var', 'median',  'std', 'size'],
        'APP_ANNUTY_TO_CRED' : ['mean', 'sum', 'min', 'max', 'median', 'std', 'size']
    }

    # Previous applications categorical features
cat_aggregations = {}
for cat in cat_cols:
    cat_aggregations[cat] = ['count']

prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations}).reset_index()
prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
prev_agg['SK_ID_CURR']= prev_agg['PREV_SK_ID_CURR_']


    # Previous Applications: Approved Applications - only numerical features
approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations).reset_index()
approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])

    # Previous Applications: Refused Applications - only numerical features
refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations).reset_index()
refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])

refused_agg['SK_ID_CURR']= refused_agg['REFUSED_SK_ID_CURR_']
approved_agg['SK_ID_CURR']= approved_agg['APPROVED_SK_ID_CURR_']
prev = prev_agg.merge(refused_agg, how='left', on='SK_ID_CURR')
prev = prev_agg.merge(approved_agg, how='left', on='SK_ID_CURR')

In [ ]:
prev=prev.fill

In [62]:
t = pd.read_csv('application_train.csv')
target = t[['SK_ID_CURR', 'TARGET']]
prev =prev.merge(target, how='left', on='SK_ID_CURR')
new=prev[prev['TARGET'].notnull()]
new=new.fillna(0)
del new['PREV_SK_ID_CURR_']
del new['APPROVED_SK_ID_CURR_']
del new['PREV_AMT_ANNUITY_MEAN']
new = new.replace([np.nan, -np.nan], 0)
new=new.replace([np.inf, -np.inf], 0)
new=abs(new)

In [68]:
# Выявяет самые низкие корреляции
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
select = SelectKBest(chi2, k=20)
select.fit(new, new['TARGET'])
# Get idxs of columns to keep
mask = select.get_support()
new_features = new.columns[mask]
# Create new dataframe with only desired columns, or overwrite existing
features_dataframe_new = new[new_features]
#features_dataframe_new.corr()
neew=pd.DataFrame(prev[new_features],prev['SK_ID_CURR']).reset_index()

In [71]:
neew.to_csv('pprev.csv')

del prev['PREV_AMT_CREDIT_MEDIAN']
del prev['PREV_CNT_PAYMENT_MEAN']
del prev['PREV_CNT_PAYMENT_SUM']
del prev['PREV_NAME_CONTRACT_TYPE_Consumer loans_COUNT']
del prev['PREV_NAME_CONTRACT_TYPE_Revolving loans_COUNT']
del prev['PREV_NAME_CONTRACT_TYPE_XNA_COUNT']
del prev['PREV_FLAG_LAST_APPL_PER_CONTRACT_N_COUNT']
del prev['PREV_FLAG_LAST_APPL_PER_CONTRACT_Y_COUNT']
prev=prev.fillna(0)
del prev['PREV_SK_ID_CURR_']
del prev['APPROVED_SK_ID_CURR_']
del prev['PREV_AMT_ANNUITY_MEAN']

#Выявляет самую сильную корреляцию

from sklearn.feature_selection import SelectKBest, f_classif
# Create and fit selector
selector = SelectKBest(f_classif, k=10)
selector.fit(prev, prev['PREV_NAME_CONTRACT_STATUS_Approved_COUNT'])
# Get idxs of columns to keep
mask = selector.get_support()
new_features = prev.columns[mask]
# Create new dataframe with only desired columns, or overwrite existing
features_dataframe_new = prev[new_features]

# Выявяет самые низкие корреляции
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
select = SelectKBest(chi2, k=5)
select.fit(prev, prev['PREV_NAME_CONTRACT_STATUS_Approved_COUNT'])
# Get idxs of columns to keep
mask = select.get_support()
new_features = prev.columns[mask]
# Create new dataframe with only desired columns, or overwrite existing
features_dataframe_new = prev[new_features]

features_dataframe_new.corr()

In [70]:
CCB = pd.read_csv('credit_card_balance.csv')

In [126]:
def m(DPD):
    
    # DPD is a series of values of SK_DPD for each of the groupby combination 
    # We convert it to a list to get the number of SK_DPD values NOT EQUALS ZERO
    x = DPD.tolist()
    c = 0
    for i,j in enumerate(x):
        if j != 0:
            c += 1
    
    return c 

#% of MINIMUM PAYMENTS MISSED

def f(min_pay, total_pay):
    
    M = min_pay.tolist()
    T = total_pay.tolist()
    P = len(M)
    c = 0 
    # Find the count of transactions when Payment made is less than Minimum Payment 
    for i in range(len(M)):
        if T[i] < M[i]:
            c += 1  
    return (100*c)/P


#amt balance
def amt(x1, x2):
    
    balance = x1.max()
    limit = x2.max()
    
    return (balance/limit)

#active credit
def ac(x):
    if x == 'Closed':
        y = 0
    else:
        y = 1    
    return y

# days_cred
def days_cred(x):
    if x<0:
        y = 0
    else:
        y = 1   
    return y

In [76]:
# No of Loans per customer 

grp = CCB.groupby(by = ['SK_ID_CURR'])['SK_ID_PREV'].nunique().reset_index().rename(index = str, columns = {'SK_ID_PREV': 'NO_LOANS'})
CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')

# No of Installments paid per Loan per Customer 

grp = CCB.groupby(by = ['SK_ID_CURR', 'SK_ID_PREV'])['CNT_INSTALMENT_MATURE_CUM'].max().reset_index().rename(index = str, columns = {'CNT_INSTALMENT_MATURE_CUM': 'NO_INSTALMENTS'})
grp1 = grp.groupby(by = ['SK_ID_CURR'])['NO_INSTALMENTS'].sum().reset_index().rename(index = str, columns = {'NO_INSTALMENTS': 'TOTAL_INSTALMENTS'})
CCB = CCB.merge(grp1, on = ['SK_ID_CURR'], how = 'left')

# Average Number of installments paid per loan 

CCB['INSTALLMENTS_PER_LOAN'] = (CCB['TOTAL_INSTALMENTS']/CCB['NO_LOANS']).astype('uint32')
del CCB['TOTAL_INSTALMENTS']
del CCB['NO_LOANS']

CCB['AMT_CREDIT_LIMIT_ACTUAL1'] = CCB['AMT_CREDIT_LIMIT_ACTUAL']

# Calculate the ratio of Amount Balance to Credit Limit - CREDIT LOAD OF CUSTOMER 
# This is done for each Credit limit value per loan per Customer 

grp = CCB.groupby(by = ['SK_ID_CURR', 'SK_ID_PREV', 'AMT_CREDIT_LIMIT_ACTUAL']).apply(lambda x: amt(x.AMT_BALANCE, x.AMT_CREDIT_LIMIT_ACTUAL1)).reset_index().rename(index = str, columns = {0: 'CREDIT_LOAD1'})
del CCB['AMT_CREDIT_LIMIT_ACTUAL1']

# We now calculate the mean Credit load of All Loan transactions of Customer 
grp1 = grp.groupby(by = ['SK_ID_CURR'])['CREDIT_LOAD1'].mean().reset_index().rename(index = str, columns = {'CREDIT_LOAD1': 'CREDIT_LOAD'})
CCB = CCB.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
del grp, grp1

#AVERAGE NUMBER OF TIMES DAYS PAST DUE HAS OCCURRED PER CUSTOMER

grp = CCB.groupby(by = ['SK_ID_CURR', 'SK_ID_PREV']).apply(lambda x: m(x.SK_DPD)).reset_index().rename(index = str, columns = {0: 'NO_DPD'})
grp1 = grp.groupby(by = ['SK_ID_CURR'])['NO_DPD'].mean().reset_index().rename(index = str, columns = {'NO_DPD' : 'DPD_COUNT'})

CCB = CCB.merge(grp1, on = ['SK_ID_CURR'], how = 'left')

#AVERAGE OF DAYS PAST DUE PER CUSTOMER

grp = CCB.groupby(by= ['SK_ID_CURR'])['SK_DPD'].mean().reset_index().rename(index = str, columns = {'SK_DPD': 'AVG_DPD'})
CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 


grp = CCB.groupby(by = ['SK_ID_CURR']).apply(lambda x: f(x.AMT_INST_MIN_REGULARITY, x.AMT_PAYMENT_CURRENT)).reset_index().rename(index = str, columns = { 0 : 'PERCENTAGE_MISSED_PAYMENTS'})
CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')

# RATIO OF CASH VS CARD SWIPES

grp = CCB.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_ATM_CURRENT'].sum().reset_index().rename(index = str, columns = {'AMT_DRAWINGS_ATM_CURRENT' : 'DRAWINGS_ATM'})
CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = CCB.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_CURRENT'].sum().reset_index().rename(index = str, columns = {'AMT_DRAWINGS_CURRENT' : 'DRAWINGS_TOTAL'})
CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

CCB['CASH_CARD_RATIO1'] = (CCB['DRAWINGS_ATM']/CCB['DRAWINGS_TOTAL'])*100
del CCB['DRAWINGS_ATM']
del CCB['DRAWINGS_TOTAL']
gc.collect()

grp = CCB.groupby(by = ['SK_ID_CURR'])['CASH_CARD_RATIO1'].mean().reset_index().rename(index = str, columns ={ 'CASH_CARD_RATIO1' : 'CASH_CARD_RATIO'})
CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

del CCB['CASH_CARD_RATIO1']
gc.collect()

#AVERAGE DRAWING PER CUSTOMER

grp = CCB.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_CURRENT'].sum().reset_index().rename(index = str, columns = {'AMT_DRAWINGS_CURRENT' : 'TOTAL_DRAWINGS'})
CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = CCB.groupby(by = ['SK_ID_CURR'])['CNT_DRAWINGS_CURRENT'].sum().reset_index().rename(index = str, columns = {'CNT_DRAWINGS_CURRENT' : 'NO_DRAWINGS'})
CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

CCB['DRAWINGS_RATIO1'] = (CCB['TOTAL_DRAWINGS']/CCB['NO_DRAWINGS'])*100
del CCB['TOTAL_DRAWINGS']
del CCB['NO_DRAWINGS']
gc.collect()

grp = CCB.groupby(by = ['SK_ID_CURR'])['DRAWINGS_RATIO1'].mean().reset_index().rename(index = str, columns ={ 'DRAWINGS_RATIO1' : 'DRAWINGS_RATIO'})
CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

del CCB['DRAWINGS_RATIO1']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  


In [88]:
CCB.to_csv('CCB.csv')

In [25]:
bureau = pd.read_csv('bureau.csv')

In [33]:
# Number of Loans per Customer
NumLoanPerCust = bureau[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT'].count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'BUREAU_LOAN_COUNT'})
# Number of types of Credit loans for each Customer 
NumTypeofLoansPerCust = bureau[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})

bureau = bureau.merge(NumLoanPerCust, on = ['SK_ID_CURR'], how = 'left')
bureau = bureau.merge(NumTypeofLoansPerCust, on = ['SK_ID_CURR'], how = 'left')

# Average Number of Loans per Loan Type
bureau['AVERAGE_LOAN_TYPE'] = bureau['BUREAU_LOAN_COUNT']/bureau['BUREAU_LOAN_TYPES']
bureau['CREDIT_ACTIVE_BINARY'] = bureau['CREDIT_ACTIVE']
bureau['CREDIT_ACTIVE_BINARY'] = bureau.apply(lambda x: ac(x.CREDIT_ACTIVE), axis = 1)

# Calculate mean number of loans that are ACTIVE per CUSTOMER 
MeanNumofLoanActiveCust = bureau.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'ACTIVE_LOANS_PERCENTAGE'})

# Groupby each Customer and Sort values of DAYS_CREDIT in ascending order
DaysCredit = bureau[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])
DaysCredit2 = DaysCredit.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False)).reset_index(drop = True)


# Calculate Difference between the number of Days 
DaysCredit2['DAYS_CREDIT1'] = DaysCredit2['DAYS_CREDIT']*-1
DaysCredit2['DAYS_DIFF'] = DaysCredit2.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT1'].diff()
DaysCredit2['DAYS_DIFF'] = DaysCredit2['DAYS_DIFF'].fillna(0).astype('uint32')

bureau['CREDIT_ENDDATE_BINARY'] = bureau['DAYS_CREDIT_ENDDATE']
bureau['CREDIT_ENDDATE_BINARY'] = bureau.apply(lambda x: days_cred(x.DAYS_CREDIT_ENDDATE), axis = 1)

#% of LOANS PER CUSTOMER WHERE END DATE FOR CREDIT IS PAST
LoansEndDate = bureau.groupby(by = ['SK_ID_CURR'])['CREDIT_ENDDATE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ENDDATE_BINARY': 'CREDIT_ENDDATE_PERCENTAGE'})


bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)
bureau['AMT_CREDIT_SUM'] = bureau['AMT_CREDIT_SUM'].fillna(0)

SumCredDebt = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
SumCredDebt2 = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM': 'TOTAL_CUSTOMER_CREDIT'})

bureau = bureau.merge(SumCredDebt, on = ['SK_ID_CURR'], how = 'left')
bureau = bureau.merge(SumCredDebt2, on = ['SK_ID_CURR'], how = 'left')

bureau['DEBT_CREDIT_RATIO'] = bureau['TOTAL_CUSTOMER_DEBT']/bureau['TOTAL_CUSTOMER_CREDIT']
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)
bureau['AMT_CREDIT_SUM_OVERDUE'] = bureau['AMT_CREDIT_SUM_OVERDUE'].fillna(0)

AMT_CREDIT_SUM_DEBT = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
AMT_CREDIT_SUM_OVERDUE = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_OVERDUE']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_OVERDUE'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_OVERDUE': 'TOTAL_CUSTOMER_OVERDUE'})

bureau = bureau.merge(AMT_CREDIT_SUM_DEBT, on = ['SK_ID_CURR'], how = 'left')
bureau = bureau.merge(AMT_CREDIT_SUM_OVERDUE, on = ['SK_ID_CURR'], how = 'left')
bureau['OVERDUE_DEBT_RATIO'] = bureau['TOTAL_CUSTOMER_OVERDUE_y']/bureau['TOTAL_CUSTOMER_DEBT']

CNT_CREDIT_PROLONG = bureau[['SK_ID_CURR', 'CNT_CREDIT_PROLONG']].groupby(by = ['SK_ID_CURR'])['CNT_CREDIT_PROLONG'].mean().reset_index().rename( index = str, columns = { 'CNT_CREDIT_PROLONG': 'AVG_CREDITDAYS_PROLONGED'})
bureau = bureau.merge(CNT_CREDIT_PROLONG, on = ['SK_ID_CURR'], how = 'left')

In [65]:
del bureau['CREDIT_ENDDATE_BINARY'], bureau['BUREAU_LOAN_COUNT'], bureau['BUREAU_LOAN_TYPES'], \
bureau['CREDIT_ACTIVE_BINARY'] , bureau['DAYS_CREDIT_ENDDATE'],bureau['TOTAL_CUSTOMER_DEBT'], \
bureau['TOTAL_CUSTOMER_CREDIT']

In [66]:
gc.collect()

1265

In [89]:
bureau.to_csv('bureau_new.csv')

In [49]:
# Calculate the Difference in ENDDATES and fill missing values with zero 
DaysCredit2['DAYS_ENDDATE_DIFF'] = DaysCredit2.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT_ENDDATE1'].diff()
DaysCredit2['DAYS_ENDDATE_DIFF'] = DaysCredit2['DAYS_ENDDATE_DIFF'].fillna(0).astype('uint32')

bureau = bureau.merge(DaysCredit2, on = ['SK_ID_CURR'], how = 'left')

AvgDaysEnddateDiff = bureau[['SK_ID_CURR', 'DAYS_ENDDATE_DIFF']].groupby(by = ['SK_ID_CURR'])['DAYS_ENDDATE_DIFF'].mean().reset_index().rename( index = str, columns = {'DAYS_ENDDATE_DIFF': 'AVG_ENDDATE_FUTURE'})




KeyError: 'Column not found: DAYS_CREDIT_ENDDATE1'

In [13]:
pos, cat_cols = one_hot_encoder(pos)
# Features
aggregations = {
        'MONTHS_BALANCE': ['size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
for cat in cat_cols:
    aggregations[cat] = ['mean']
    
pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
# Count pos cash accounts
pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
pos=pos_agg.reset_index().copy()

del pos['POS_NAME_CONTRACT_STATUS_Demand_MEAN']
del pos['POS_NAME_CONTRACT_STATUS_Canceled_MEAN']
del pos['POS_NAME_CONTRACT_STATUS_Approved_MEAN']
del pos['POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN']
del pos['POS_NAME_CONTRACT_STATUS_XNA_MEAN']
# del features with high correlation
del pos['POS_SK_DPD_MEAN']
del pos['POS_MONTHS_BALANCE_MEAN']
del pos['POS_NAME_CONTRACT_STATUS_Completed_MEAN']
del pos['POS_COUNT']
del pos['POS_SK_DPD_MEAN']
del pos['POS_SK_DPD_DEF_MEAN']

In [108]:
pos.to_csv('new_pos.csv')

In [14]:
ins, cat_cols = one_hot_encoder(ins)
    # Percentage and difference paid in each installment (amount paid and installment value)
ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
# Days past due and days before due (no negative values)
ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
# Features: Perform aggregations
aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max'],
        'DBD': ['max', 'mean'],
        'PAYMENT_PERC': ['var'],
        'PAYMENT_DIFF': ['sum'],
        'DAYS_ENTRY_PAYMENT': ['sum']
    }
for cat in cat_cols:
    aggregations[cat] = ['mean']
ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
# Count installments accounts
ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
ins=ins_agg.reset_index().copy()

In [113]:
ins.to_csv('ins.csv')

In [117]:
ins.head()

,SK_ID_CURR,INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE,INSTAL_DPD_MAX,INSTAL_DBD_MAX,INSTAL_DBD_MEAN,INSTAL_PAYMENT_PERC_VAR,INSTAL_PAYMENT_DIFF_SUM,INSTAL_DAYS_ENTRY_PAYMENT_SUM,INSTAL_COUNT
0,100001,2,11.0,36.0,8.857143,0.0,0.0,-15365.0,7
1,100002,2,0.0,31.0,20.421053,0.0,0.0,-5993.0,19
2,100003,2,0.0,14.0,7.160000,0.0,0.0,-34633.0,25
3,100004,2,0.0,11.0,7.666667,0.0,0.0,-2285.0,3
4,100005,2,1.0,37.0,23.666667,0.0,0.0,-5486.0,9


In [125]:
ins['INSTAL_COUNT'].nunique()

323

In [5]:
df=pd.read_csv('train123.csv')
prev=pd.read_csv('pprev.csv')
cc=pd.read_csv('CCB.csv')
bureau_new=pd.read_csv('bureau_new.csv')
pos=pd.read_csv('new_pos.csv')
ins=pd.read_csv('ins.csv')

df = df.merge(prev, how='left', on='SK_ID_CURR')
df = df.merge(pos, how='left', on='SK_ID_CURR')
df = df.merge(cc, how='left', on='SK_ID_CURR')
df = df.merge(ins, how='left', on='SK_ID_CURR')
df = df.merge(bureau, how='left', on='SK_ID_CURR')

In [ ]:
df = df.merge(prev, how='left', on='SK_ID_CURR')
df = df.merge(pos, how='left', on='SK_ID_CURR')
df = df.merge(cc, how='left', on='SK_ID_CURR')
df = df.merge(ins, how='left', on='SK_ID_CURR')
df = df.merge(bureau, how='left', on='SK_ID_CURR')

In [ ]:
df.head()

In [22]:
df.to_csv('new_full_dataset.csv')

In [ ]:
test = train[train['TARGET'].isnull()]
train1= train[train['TARGET'] ==  1]
train2= train[train['TARGET'] ==  0]
train=pd.concat([train1, train2])

In [ ]:
cat_feat = list(train.dtypes[train.dtypes == object].index)
num_feat = [f for f in train if f not in cat_feat ]

def get_woe_v1(df_train, df_test, col, target_col):
    all_good = len(df_train[df_train[target_col] == 1][col])
    all_bad = len(df_train[df_train[target_col] == 0][col])
    odds_series = (
        df_train[df_train[target_col] == 1][col].value_counts()
        /
        df_train[df_train[target_col] == 0][col].value_counts()
    )
    odds_series = odds_series / all_good * all_bad
    category_woe_dict = np.log(odds_series).to_dict()
    df_train[col + '_woe'] = df_train[col].apply(category_woe_dict.get)
    df_test[col + '_woe'] = df_test[col].apply(category_woe_dict.get)
    return df_train, df_test


for cat_feat in cat_feat:
    data_train, data_test = get_woe_v1(train, test, cat_feat, 'TARGET')
    
cat_feat = list(data_train.dtypes[data_train.dtypes == object].index)
num_feat = [f for f in train if f not in cat_feat ]

data_train=data_train.drop(cat_feat, axis=1)
data_test=data_test.drop(cat_feat, axis=1)

In [ ]:
from sklearn.preprocessing import Imputer
imp=Imputer(missing_values='NaN', strategy='median', axis=1) 
imp.fit(data_train)
data_train = imp.fit_transform(data_train)
data_test = imp.fit_transform(data_test)

#data_test = data_test.replace([np.nan, -np.nan], 0)
#data_train = data_train.replace([np.nan, -np.nan], 0)
#data_train=data_train.replace([np.inf, -np.inf], 0)
#data_test=data_test.replace([np.inf, -np.inf], 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
m=scaler.fit(data_train)
X_train=scaler.transform(data_train)
n=scaler.fit(data_test)
X_test=scaler.transform(data_test)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn import tree

In [ ]:
clf_ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5, min_samples_leaf=20, max_features=0.8),
                             n_estimators=20, learning_rate=0.1)
clf_ada.fit(X_train, y_train)

In [ ]:
y_pred=clf_ada.predict_proba(X_test)
sample_submission=pd.read_csv('sample_submission-9.csv')
sample_submission['TARGET']=y_pred
sample_submission.to_csv('sample_submission.csv', index=False)

cc, cat_cols = one_hot_encoder(cc)
# General aggregations
cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
cc_agg = cc.groupby('SK_ID_CURR').agg(['mean', 'count'])
cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
# Count credit card lines
cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
cc=cc_agg.reset_index().copy()

bb, bb_cat = one_hot_encoder(bb)
bureau, bureau_cat = one_hot_encoder(bureau)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
for col in bb_cat:
    bb_aggregations[col] = ['mean']
bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
#bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)

num_aggregations = {
        'DAYS_CREDIT': ['mean'],
        'DAYS_CREDIT_ENDDATE': ['max'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_ANNUITY': ['mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['sum']
    }
    # Bureau and bureau_balance categorical features
cat_aggregations = {}
for cat in bureau_cat: cat_aggregations[cat] = ['mean']
for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    
active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])

# Bureau: Closed credits - using only numerical aggregations
closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])

bureau_agg=bureau_agg.reset_index()
active_agg=active_agg.reset_index()
closed_agg=closed_agg.reset_index()
bureau_agg = bureau_agg.merge(closed_agg, how='left', on='SK_ID_CURR')
bureau = bureau_agg.merge(active_agg, how='left', on='SK_ID_CURR')

train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

if stratified:
    folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
else:
    folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
# Create arrays and dataframes to store results
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]



for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
    train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]
    
    clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1 )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 100, early_stopping_rounds= 200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        test_df['TARGET'] = sub_preds
        test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
    display_importances(feature_importance_df)
    return feature_importance_df

# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')